In [48]:
import torch
import numpy as np
import itertools

#デフォルトボックスを出力するクラス
class DBox(object):
    def __init__(self, cfg):
        super(DBox, self).__init__()
        
        #初期設定
        self.image_size = cfg['input_size'] #画像サイズの300
        #[38, 19, ...]各sourceの特徴量マップのサイズ
        self.feature_maps = cfg['feature_maps']
        self.num_priors = len(cfg['feature_maps']) #sourceの個数=6
        self.steps = cfg['steps'] #[8, 16, ...] DBoxのピクセルサイズ(特徴マップの1つの大きさ)
        self.min_sizes = cfg['min_sizes'] #[30,60,...]小さい正方形のDBoxのピクセルサイズ
        self.max_sizes = cfg['max_sizes'] #[60,111,...]大きい正方形のDBoxのピクセルサイズ
        self.aspect_ratios = cfg['aspect_ratios'] #長方形のDBoxのアスペクト比
        

    def make_dbox_list(self):
        '''DBoxを作成する'''
        mean =[]
        #'feature_maps' : [38,19,10,5,3,1]
        for k, f in enumerate(self.feature_maps):
            for i, j in itertools.product(range(f), repeat=2): #fまでの数で2ペアの組み合わせを作る
                
                #特徴量の画像サイズ
                #300 / 'steps':[8,16,32, 64, 100, 300], 
                f_k = self.image_size / self.steps[k]
                
                #DBoxの中心座標x,y　ただし0~1で規格化している
                cx = (j + 0.5) / f_k
                cy = (i + 0.5) / f_k
                
                #アスペクト比1の小さいDBox[cx,cy, width, height]
                #'min_sizes':[30,60,111,162,213,264]
                s_k = self.min_sizes[k]/self.image_size
                mean += [cx, cy, s_k, s_k]
                
                #アスペクト比1の大きいDBox [cx, cy, width, height]
                #'max_size': [60, 111, 162, 213, 264, 315],
                s_k_prime =  np.sqrt(s_k * (self.max_sizes[k]/self.image_size))
                mean += [cx, cy, s_k_prime, s_k_prime]
                
                #その他のアスペクト比のdefBox[cx, cy, width, height]
                for ar in self.aspect_ratios[k]:
                    mean += [cx, cy, s_k*np.sqrt(ar), s_k/np.sqrt(ar)]
                    mean += [cx, cy, s_k/np.sqrt(ar), s_k*np.sqrt(ar)]
                    
        #DBoxをテンソルに変換torch.Size([8732, 4])
        output = torch.Tensor(mean).view(-1, 4)
            
        #DBoxが画像の外にはみ出るのを防ぐため、大きさを最小0　最大1にする
        output.clamp_(max=1, min=0)
            
        return output
            

In [49]:
import pandas as pd
#動作確認

#SSD300の設定
ssd_cfg = {
    'num_classes': 21,  #背景クラスを含めた合計クラス
    'input_size': 300,  #画像の入力サイズ
    'bbox_aspect_num': [4,6,6,6,4,4],  #各sourceに対して、出力されるデフォルトボックスの数
    'feature_maps': [38, 19, 10, 5, 3, 1], ##各sourceの大きさを決める
    'steps': [8, 16, 32, 64, 100, 300], #DBoxの(元画像の入力サイズ上で見た時の)大きさを決める
    'min_sizes': [30, 60, 111, 162, 213, 264], #小さい正方形のDBoxの大きさを決める
    'max_sizes': [60, 111, 162, 213, 264, 315], #大きい正方形のDBoxの大きさを決める
    'aspect_ratios': [[2], [2,3], [2,3], [2,3], [2], [2]],
}

#DBoxの作成
dbox = DBox(ssd_cfg)
dbox_list = dbox.make_dbox_list()


#DBoxの出力を確認する
pd.DataFrame(dbox_list.numpy())

,0,1,2,3
0,0.013333,0.013333,0.100000,0.100000
1,0.013333,0.013333,0.141421,0.141421
2,0.013333,0.013333,0.141421,0.070711
3,0.013333,0.013333,0.070711,0.141421
4,0.040000,0.013333,0.100000,0.100000
...,...,...,...,...
8727,0.833333,0.833333,0.502046,1.000000
8728,0.500000,0.500000,0.880000,0.880000
8729,0.500000,0.500000,0.961249,0.961249
8730,0.500000,0.500000,1.000000,0.622254
